In [ ]:
pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
pip install moviepy

In [ ]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5dg4lynn
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5dg4lynn
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
pip install transformers

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
pip install --upgrade pytube


In [ ]:
pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.0 MB/s eta 0:00:00


In [ ]:
import os
from yt_dlp import YoutubeDL
from moviepy.editor import AudioFileClip
import whisper
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
import torch

class VideoSummarizer:
    def __init__(self):
        # Initialize transcription model
        try:
            self.transcription_model = whisper.load_model("base")
        except Exception as e:
            print("Error loading Whisper model:", e)
            self.transcription_model = None

        # Initialize summarization model and tokenizer
        try:
            self.summarization_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
            self.tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
        except Exception as e:
            print("Error loading Summarization model:", e)
            self.summarization_model = None
            self.tokenizer = None

    def download_youtube_video(self, url, output_path="videos"):
        """Download a YouTube video using yt-dlp and save it to the output path."""
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        }
        with YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            file_path = ydl.prepare_filename(info_dict)
            print(f"Video downloaded to {file_path}")
            return file_path.replace('.webm', '.mp3').replace('.m4a', '.mp3')  # Ensure file extension matches

    def extract_audio(self, video_path, output_path="audios"):
        """Extract audio from a video file and save it as a .wav file."""
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        audio_path = os.path.join(output_path, os.path.splitext(os.path.basename(video_path))[0] + ".wav")
        audio_clip = AudioFileClip(video_path)
        audio_clip.write_audiofile(audio_path, codec="pcm_s16le")
        audio_clip.close()
        print(f"Audio extracted to {audio_path}")
        return audio_path

    def transcribe_audio(self, audio_path):
        """Transcribe audio using OpenAI's Whisper model."""
        if self.transcription_model is None:
            raise RuntimeError("Transcription model not loaded.")
        print("Transcription in progress...")
        result = self.transcription_model.transcribe(audio_path)
        print("Transcription complete.")
        return result['text']

    def summarize_text(self, text):
        """Summarize the given text using a pre-trained BART model."""
        if self.summarization_model is None or self.tokenizer is None:
            raise RuntimeError("Summarization model or tokenizer not loaded.")
        print("Summarization in progress...")
        inputs = self.tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = self.summarization_model.generate(
            inputs["input_ids"],
            max_length=150,
            min_length=50,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True,
        )
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print("Summarization complete.")
        return summary

    def process_video(self, youtube_url):
        """Complete pipeline: Download video, extract audio, transcribe, and summarize."""
        video_path = self.download_youtube_video(youtube_url)
        audio_path = self.extract_audio(video_path)
        transcript = self.transcribe_audio(audio_path)

        # Summarize the transcript
        summary = self.summarize_text(transcript)

        # Save transcript and summary to files
        output_dir = "output"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        base_name = os.path.splitext(os.path.basename(video_path))[0]
        transcript_file = os.path.join(output_dir, base_name + "_transcript.txt")
        summary_file = os.path.join(output_dir, base_name + "_summary.txt")

        with open(transcript_file, "w", encoding="utf-8") as f:
            f.write(transcript)
        print(f"Transcript saved to {transcript_file}")

        with open(summary_file, "w", encoding="utf-8") as f:
            f.write(summary)
        print(f"Summary saved to {summary_file}")

        return summary

# Example usage
if __name__ == "__main__":
    youtube_url = input("Enter the YouTube video URL: ")
    summarizer = VideoSummarizer()
    summary = summarizer.process_video(youtube_url)
    print("\nFinal Summary:\n")
    print(summary)


  if event.key is 'enter':

